<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_sentence_similarity_vector_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#句子相似度检测，对每个句子都进行打分，而不是只对单个句子对打分

In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U evaluate
import evaluate
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,DataCollatorWithPadding,Trainer,TrainingArguments,pipeline
from transformers import BertForSequenceClassification, BertPreTrainedModel, BertModel
from typing import Optional
from transformers.configuration_utils import PretrainedConfig
import torch
from torch.nn import CosineSimilarity, CosineEmbeddingLoss

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


In [2]:
datasets = load_dataset("clue",name="afqmc")
#针对“对每个句子都进行打分"的任务，model需要自己创建，所以这里先不AutoModelFor进行引入了
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/3861 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/34334 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4316 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
datasets

DatasetDict({
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 4316
    })
})

In [4]:
train_dataset=datasets["train"]
train_dataset[0]

{'sentence1': '蚂蚁借呗等额还款可以换成先息后本吗',
 'sentence2': '借呗有先息到期还本吗',
 'label': 0,
 'idx': 0}

In [5]:
#需要构建句子对的格式，也就是说：原来input_ids就是一个一维列表，每个句子对都合在了一起，tokentype_ids有0有1[0,0...0,1,1,..1]
#但是现在我们需要inputids是一个二维列表[[sentence_A_ids],[sentence_B_ids]],而tokentypeids则是[[0,0,..0],[0,0,.0]]
def data_process(examples):
  sentences = []
  labels = []
  for A,B,label in zip(examples["sentence1"],examples["sentence2"],examples["label"]):
    sentences.append(A)
    sentences.append(B)
    labels.append(1 if label==1 else -1)
    #因为后面要用CosineEmbeddingLoss：a criterion that measures the loss given input tensors
    # :math:`x_1`, :math:`x_2` and a `Tensor` label :math:`y` with values 1 or -1.
  #批量处理后，sentences就会是[A1,B1,A2,B2,A3,B3,...]
  #labels就是每个句子对所对应的label
  tokenized_examples=tokenizer(sentences,max_length=128,truncation=True,padding="max_length")
  #因为这里已经有padding了，所以这次不需要datacollatorforpadding了
  #tokenized_examples是{input_ids:[[A1],[B1],[A2],[B2]...],token_type_ids:[[A1],[B1],[A2],[B2]...],attention_mask:[[A1],[B1],[A2],[B2]...]}
  #需要把句子对合到同一个list里：
  tokenized_examples={k:[v[i:i+2]for i in range(0,len(v),2)] for k, v in tokenized_examples.items()}
  tokenized_examples["labels"] = labels
  return tokenized_examples

tokenized_datasets=datasets.map(data_process,batched=True,remove_columns=datasets["train"].column_names)

tokenized_datasets

Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Map:   0%|          | 0/34334 [00:00<?, ? examples/s]

Map:   0%|          | 0/4316 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4316
    })
})

In [6]:
tokenized_datasets["train"][0]

{'input_ids': [[101,
   6010,
   6009,
   955,
   1446,
   5023,
   7583,
   6820,
   3621,
   1377,
   809,
   2940,
   2768,
   1044,
   2622,
   1400,
   3315,
   1408,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [101,
   955,
   1446,
   3300,
   1044,
   2622,
   1168,
   3309,
   6820,
   3315,
   1408,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,


In [7]:
#创建模型，参考BertForSequenceClassification
#继承 BertPreTrainedModel是为了调用模型的时候可以使用.from_pretrained来调用
class BertForVectorMatch(BertPreTrainedModel):
  def __init__(self, config):
      super().__init__(config)
      self.config = config
      self.bert = BertModel(config)
      # Initialize weights and apply final processing
      self.post_init()
  def forward(
  self,
  input_ids: Optional[torch.Tensor] = None,
  attention_mask: Optional[torch.Tensor] = None,
  token_type_ids: Optional[torch.Tensor] = None,
  position_ids: Optional[torch.Tensor] = None,
  head_mask: Optional[torch.Tensor] = None,
  inputs_embeds: Optional[torch.Tensor] = None,
  labels: Optional[torch.Tensor] = None,
  output_attentions: Optional[bool] = None,
  output_hidden_states: Optional[bool] = None,
  return_dict: Optional[bool] = None,
):
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    senA_input_ids, senB_input_ids = input_ids[:, 0], input_ids[:, 1]
    senA_attention_mask, senB_attention_mask = attention_mask[:, 0], attention_mask[:, 1]
    senA_token_type_ids, senB_token_type_ids = token_type_ids[:, 0], token_type_ids[:, 1]

    senA_outputs = self.bert(
    senA_input_ids,
    attention_mask=senA_attention_mask,
    token_type_ids=senA_token_type_ids,
    position_ids=position_ids,
    head_mask=head_mask,
    inputs_embeds=inputs_embeds,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
)
    senB_outputs = self.bert(
            senB_input_ids,
            attention_mask=senB_attention_mask,
            token_type_ids=senB_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
    senA_pooled_output = senA_outputs[1]
    senB_pooled_output = senB_outputs[1]
    cos = CosineSimilarity()(senA_pooled_output, senB_pooled_output)
    loss = None
    if labels is not None:
      loss_fct = CosineEmbeddingLoss(0.3) #余弦距离<0.3就不算Loss
      loss = loss_fct(senA_pooled_output, senB_pooled_output, labels)
      #labels要么是1，要么是-1，代表同类/不同类

    output = (cos,)
    return ((loss,) + output) if loss is not None else output
model = BertForVectorMatch.from_pretrained("hfl/chinese-macbert-base")


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def metric(results):
  predictions,labels=results
  predictions=[int(p > 0.7) for p in predictions]
  labels =  [int(l > 0) for l in labels]
  acc_scores = acc.compute(predictions=predictions,references=labels)
  f1_scores = f1.compute(predictions=predictions,references=labels)
  acc_scores.update(f1_scores)
  return acc_scores

In [9]:
train_args = TrainingArguments(output_dir="./vector_match_model",      # 输出文件夹
                per_device_train_batch_size=4,  # 训练时的batch_size
                per_device_eval_batch_size=32,  # 验证时的batch_size
                logging_steps=10,                # log 打印的频率
                evaluation_strategy="steps",     # 评估策略
                eval_steps=20,
                save_steps=10,           # 保存策略
                learning_rate=2e-5,              # 学习率
                weight_decay=0.01,               # weight_decay
                metric_for_best_model="f1",      # 设定评估指标
                num_train_epochs=1)

In [10]:
# train_args = TrainingArguments(output_dir="./dual_model",      # 输出文件夹
#                                per_device_train_batch_size=32,  # 训练时的batch_size
#                                per_device_eval_batch_size=32,  # 验证时的batch_size
#                                logging_steps=10,                # log 打印的频率
#                                evaluation_strategy="epoch",     # 评估策略
#                                save_strategy="epoch",           # 保存策略
#                                save_total_limit=3,              # 最大保存数
#                                learning_rate=2e-5,              # 学习率
#                                weight_decay=0.01,               # weight_decay
#                                metric_for_best_model="f1",      # 设定评估指标
#                                load_best_model_at_end=True)     # 训练完成后

In [11]:
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  compute_metrics=metric)

In [12]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [13]:
trainer.evaluate(tokenized_datasets["test"])

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#模型预测的pipeline也要自己写，参考多选视频的评估部分

In [23]:
class SentenceSimilarityPipeline:
  def __init__(self,model,tokenizer) -> None:#类型提示（Type Hints），这是一种在函数或方法声明中指定参数类型和返回类型的方式，有助于代码的可读性和维护性。-> None表示该构造函数没有返回值。
    self.model = model.bert
    self.tokenizer = tokenizer
    self.device = self.model.device
  def preprocess(self,A,B):
    return self.tokenizer([A,B],max_length=128,truncation=True,return_tensors="pt",padding=True)
  def predict(self,inputs):
    inputs={k:v.to(self.device)for k,v in inputs.items()}
    return self.model(**inputs)[1]   #数据格式是（2,768）
  def post_process(self,logits):
    cos = CosineSimilarity()(logits[None,0,:],logits[None,1,:]).squeeze().cpu().item() #logits升维，多一个batch的维度
    #.squeeze()：这个方法是在张量中移除大小为1的维度
    #.cpu() 把张量从gpu移动到cpu
    #.item()获取张量中的单个数值
    return cos
  def __call__(self,A,B,return_vector=False):
    inputs=self.preprocess(A,B)
    logits=self.predict(inputs)
    result=self.post_process(logits)
    if return_vector:
      return result,logits
    else:
      return result

In [24]:
model = BertForVectorMatch.from_pretrained("/content/drive/MyDrive/nlp/vector_match_model/checkpoint-30")
pipe = SentenceSimilarityPipeline(model, tokenizer)

In [25]:
pipe("我喜欢北京", "明天不行")

0.6886473894119263